## 시설물 점검진단계획 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "시설물 점검진단계획 목록"
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/시설물 점검진단계획 목록/selectIoFmChckDinsPlnList.csv'

In [2]:
APIKEYLEN = len(APIKEY)

In [3]:
# 개별 시설물 점검진단계획목록 수
g_totalCount = 0

In [4]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectIoFmChckDinsPlnList'

In [5]:
# REQPARAM = targetData["요청변수"].values[0]
# REQPARAM = REQPARAM.split(",")
# PRIMARYKEY = targetData["기본키"].values[0]
# REQPARAM

In [6]:
JSONKEY = "detail1"
DUMMY = 0

In [7]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/시설물 목록/selectIoFmMngList.csv", encoding="ms949")

In [8]:
dfCombi = imsiDf.loc[:,["fcno"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

7915

In [9]:
newParam = dfCombi.fcno.drop_duplicates().tolist()

In [10]:
orgParam = []
try:
    # 기존 데이터 및 파라미터 정보 불러오기
    # 리스트로 불러오기로 변경 0425
    orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)[0]
## 파일이 없는경우
except Exception as e:
    orgParam = []
    print(e)

ptlcmnoList = list( set(newParam) - set(orgParam) )
# 모드: 0=종료 1=append 2=새로생성
mode = 2

# 업데이트 할 내용이 없으면 종료
if ptlcmnoList == []:
    mode = 0
    print("{} 정보 quit모드 {} ".format(DATANAME, mode))
    quit()
# orgParam == []
elif orgParam == []:
    mode = 2
    print("{} 정보 new 모드 {} ".format(DATANAME, mode))
# 이외에는 append 모드
else:
    mode = 1
    print("{} 정보 append 모드 {} ".format(DATANAME, mode))

[Errno 2] No such file or directory: '../output/건설사업정보시스템/시설물 점검진단계획 목록/selectIoFmChckDinsPlnList.pickle'
시설물 점검진단계획 목록 정보 new 모드 2 


## 시설물 점검진단계획 목록 전용 스크랩 함수

In [12]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os
import time

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn,inAPIKey, inApiCall, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        inAPIKeyLen = len(inAPIKey)
        while True:
            inApiCall = inApiCall + 1
            if inType == "jsonabnormal":
                time.sleep(1)
            inParam["serviceKey"] = inAPIKey[inApiCall%inAPIKeyLen]
            print("{} page scraping start apicall iter: {} / used {}".format(i,inApiCall,inParam["serviceKey"]))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata == "NODATA_ERROR"):
                        print("{} page is empty".format(i))
                        break
                    elif( jsondata == "DB_ERROR"):
                        print("DB ERROR")
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 detailList1~5까지 봐야 한다.
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)
                # fcno를 키로 기본정보와 시설물정보를 merge하기 위함
                print("fcno ",emptyPd['fcno'].values[0])
                fcno = emptyPd['fcno'].values[0]
                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
                    jsondata = response.json()["response"]["body"]["items"]
                    if( jsondata == []):
                        print("items is empty")
                    else:
#                         jsondata["index"]=[0]
                        rowData = pd.DataFrame(jsondata)
                        
                        rowData['fcno'] = fcno # 시설물 정보에 fcno칼럼을 수동으로 추가 (머지키로 활용하기 위함)
                        #emptyPd = pd.concat([emptyPd,rowData],axis=1).reindex(emptyPd.index)
                        emptyPd = pd.merge(left = emptyPd , right = rowData, on = "fcno")
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return [emptyPd,i,inApiCall]  
    except Exception as e:
            print(e)     

In [13]:

numOfRows = 1000 # 수행 성능향상을 위해 100개씩 요청하는 것으로 수정해야 함 10 -> 1000, BASEPARAM에도 추가해야 함
PAGEYN=0

In [15]:
resultDfMerged = pd.DataFrame()
resultDf = pd.DataFrame()

APICALL = 0 #추가
ptlcmnoListLen = len(ptlcmnoList)
pageList = [] #추가

for i in range(0,ptlcmnoListLen):
    apiNo = i%APIKEYLEN
#     fcnoValue = dfCombi.loc[i].fcno
    pageNo = 1
    global g_totalCount
    g_totalCount = 0
    print("i=",i," fcnoValue= ",ptlcmnoList[i])
    while True:
        BASEPARAM={"serviceKey":APIKEY[0],"pageNo": pageNo, 'fcno':ptlcmnoList[i], 'numOfRows':numOfRows, "type":"json"}
        scrapyResult = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, APIKEY,APICALL,JSONKEY, DUMMY)
        resultDf = scrapyResult[0]
        pageList.append( scrapyResult[1] )
        APICALL = scrapyResult[2]
        if resultDf.empty: # 정상 데이터가 없는 경우
            break
        else:
            resultDfMerged = resultDfMerged.append(resultDf) 
            g_totalCount -= numOfRows
            print("g_totalCount : ", g_totalCount, " numOfRows : ", numOfRows)
            if(g_totalCount <= 0):
                break
            pageNo += 1
        

        

i= 0  fcnoValue=  0C1A37819465640DE0440003BA816705
1 page scraping start apicall iter: 1 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
fcno  0C1A37819465640DE0440003BA816705
>totalCount 104
0 no pageNo
g_totalCount :  -896  numOfRows :  1000
i= 1  fcnoValue=  E673B6425ED21C52E034080020A9316A
1 page scraping start apicall iter: 2 / used 0974320D-C689-4543-8464-10DE1E5505A9
fcno  E673B6425ED21C52E034080020A9316A
>totalCount 75
0 no pageNo
g_totalCount :  -925  numOfRows :  1000
i= 2  fcnoValue=  E673B64272261C52E034080020A9316A
1 page scraping start apicall iter: 3 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
fcno  E673B64272261C52E034080020A9316A
>totalCount 47
0 no pageNo
g_totalCount :  -953  numOfRows :  1000
i= 3  fcnoValue=  649503158D713C96E054B099289CEB28
1 page scraping start apicall iter: 4 / used 0974320D-C689-4543-8464-10DE1E5505A9
fcno  649503158D713C96E054B099289CEB28
>totalCount 10
0 no pageNo
g_totalCount :  -990  numOfRows :  1000
i= 4  fcnoValue=  78752228E0D35555E0440003B

KeyboardInterrupt: 

In [16]:
resultDfMerged

,ofcnm,rutnm,fctkdcd,fctkdnm,fctcfcd,fctcfnm,fcno,fctnm,lctnm,ccwdt,dgnbgdt,dgneddt,dgconm,cwkbgdt,cwkeddt,cstcmpnm,cwknm,totctco,index,rn,chckdinsplnnovl,chckdinsbgdt,chckdinseddt,chckdinskdcdcn,chckdinsmbdtycdcn,chckdinslevlcdcn
0,광주국토관리사무소,일반국도15호선,1,교량,3,3종,0C1A37819465640DE0440003BA816705,벽송육교,전라남도 화순군 남면 벽송리,2005-10-31,1995-05-15,1996-06-03,(주)남원건설엔지니어링,2003-09-01,2005-10-31,동아건설산업(주),화순남면우회도로축조및 포장공사,"850,000",0,1,017EE76E1266585E9FA472B09B28F43C,2022-07-01,2022-12-31,정기점검-반기별,자체,육안
1,광주국토관리사무소,일반국도15호선,1,교량,3,3종,0C1A37819465640DE0440003BA816705,벽송육교,전라남도 화순군 남면 벽송리,2005-10-31,1995-05-15,1996-06-03,(주)남원건설엔지니어링,2003-09-01,2005-10-31,동아건설산업(주),화순남면우회도로축조및 포장공사,"850,000",0,2,017EE76E0DE08B1C75E0DDD367BFEBF4,2022-01-01,2022-06-30,정기점검-반기별,자체,육안
2,광주국토관리사무소,일반국도15호선,1,교량,3,3종,0C1A37819465640DE0440003BA816705,벽송육교,전라남도 화순군 남면 벽송리,2005-10-31,1995-05-15,1996-06-03,(주)남원건설엔지니어링,2003-09-01,2005-10-31,동아건설산업(주),화순남면우회도로축조및 포장공사,"850,000",0,3,017718D7FBB6B6835E71290A6196B6DF,2021-07-01,2021-12-31,정기점검-반기별,자체,육안
3,광주국토관리사무소,일반국도15호선,1,교량,3,3종,0C1A37819465640DE0440003BA816705,벽송육교,전라남도 화순군 남면 벽송리,2005-10-31,1995-05-15,1996-06-03,(주)남원건설엔지니어링,2003-09-01,2005-10-31,동아건설산업(주),화순남면우회도로축조및 포장공사,"850,000",0,4,017718E2A9818A9C875E980F3CA540F2,2021-01-01,2021-06-30,정기점검-반기별,자체,육안
4,광주국토관리사무소,일반국도15호선,1,교량,3,3종,0C1A37819465640DE0440003BA816705,벽송육교,전라남도 화순군 남면 벽송리,2005-10-31,1995-05-15,1996-06-03,(주)남원건설엔지니어링,2003-09-01,2005-10-31,동아건설산업(주),화순남면우회도로축조및 포장공사,"850,000",0,5,01709EC161D48D2574B4EF5297C8ACA4,2020-07-01,2020-12-31,정기점검-반기별,자체,육안
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,진영국토관리사무소,일반국도25호선,1,교량,0,기타,B88280942B531D1EE0440003BA816705,비음교,경상남도 창원시 성산구 대방동,2004-02-29,1996-12-28,1997-12-31,(주)남원건설엔지니어링,1997-12-30,2004-02-29,SK건설(주)외 9개사,국도25호선 국도 대체우회도로(천선-용동) 개설공사,0,0,18,FC76755AA98D212EE044B099289CEB28,2014-06-12,2014-06-30,정기점검-반기별,자체,육안
18,진영국토관리사무소,일반국도25호선,1,교량,0,기타,B88280942B531D1EE0440003BA816705,비음교,경상남도 창원시 성산구 대방동,2004-02-29,1996-12-28,1997-12-31,(주)남원건설엔지니어링,1997-12-30,2004-02-29,SK건설(주)외 9개사,국도25호선 국도 대체우회도로(천선-용동) 개설공사,0,0,19,ECACD4FB6ED42806E0440003BA816706,2013-10-21,2013-11-05,정기점검-반기별,자체,육안
19,진영국토관리사무소,일반국도25호선,1,교량,0,기타,B88280942B531D1EE0440003BA816705,비음교,경상남도 창원시 성산구 대방동,2004-02-29,1996-12-28,1997-12-31,(주)남원건설엔지니어링,1997-12-30,2004-02-29,SK건설(주)외 9개사,국도25호선 국도 대체우회도로(천선-용동) 개설공사,0,0,20,DCA88603FBC00B80E0440003BA816705,2013-07-01,2013-12-31,정기점검-반기별,자체,육안
20,진영국토관리사무소,일반국도25호선,1,교량,0,기타,B88280942B531D1EE0440003BA816705,비음교,경상남도 창원시 성산구 대방동,2004-02-29,1996-12-28,1997-12-31,(주)남원건설엔지니어링,1997-12-30,2004-02-29,SK건설(주)외 9개사,국도25호선 국도 대체우회도로(천선-용동) 개설공사,0,0,21,EAF7F1EC85331A71E0440003BA816706,2013-05-13,2013-05-29,정기점검-반기별,자체,육안


In [97]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME,mode)

시설물 점검진단계획 목록 save compled


In [98]:
resultDfMerged.shape

(166155, 26)

In [17]:
# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
ptlcmnoList = orgParam + ptlcmnoList

paramList = [ptlcmnoList,pageList]
# 최종 파라미터 저장
cf.saveparam(paramList, SITENAME,DATANAME,SERVICENAME)